In [49]:
import tensorflow as tf
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input,Dense, Subtract, Add, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from keras.constraints import maxnorm

real_dem = pd.read_excel('C:/Users/ERIC/Desktop/RNN_Demand/Demanda Real Balance_0_v3 Dia Operacion 2020-03-09 v2020 03 23_12 25 01.xlsm',0)

# Training dataset
cen_x = real_dem.iloc[:, 3:8].values
cen_y = real_dem.iloc[:, 8:9].values

#Splitting the data into a 80/20 proportion
cen_x_train, cen_x_test, cen_y_train, cen_y_test = train_test_split(cen_x, cen_y, test_size=0.2, random_state=0)

#Standarization of the data
sc = StandardScaler()
cen_x_train = sc.fit_transform(cen_x_train)
cen_x_test = sc.fit_transform(cen_x_test)

# Input from the current day
input_tensor = Input(shape=(cen_x.shape[1],))

hidden_layer = Dense(100, activation = 'relu',kernel_constraint=maxnorm(3),#Make better results with Dropout
                     kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),#To avoid Over-fitting
                     activity_regularizer=regularizers.l1(0.001))(input_tensor) #Regularize the wieghts to have better accuracy
layer_drop = Dropout(0.5)
hidden_layer1 = Dense(50, activation = 'relu',kernel_constraint=maxnorm(3),
                      kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),
                      activity_regularizer=regularizers.l1(0.001))(hidden_layer)
layer_drop2 = Dropout(0.5)
hidden_layer2 = Dense(100, activation = 'relu', kernel_constraint=maxnorm(3),
                      kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),
                      activity_regularizer=regularizers.l1(0.001))(hidden_layer1)
layer_drop3 = Dropout(0.5)
hidden_layer3 = Dense(25, activation = 'relu', kernel_constraint=maxnorm(3), 
                      kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01),
                      activity_regularizer=regularizers.l1(0.001))(hidden_layer2)
output_tensor = Dense(1)(hidden_layer3)

model = Model(input_tensor, output_tensor)

model.compile(optimizer= 'adam',  loss='mean_absolute_error', metrics = ['mean_absolute_error'])

model.summary()

model.fit(cen_x_train,
          cen_y_train,
          epochs=100,
          batch_size=3,
          validation_split=0.1,
          verbose=True)


Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_113 (Dense)            (None, 100)               600       
_________________________________________________________________
dense_114 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_115 (Dense)            (None, 100)               5100      
_________________________________________________________________
dense_116 (Dense)            (None, 25)                2525      
_________________________________________________________________
dense_117 (Dense)            (None, 1)                 26        
Total params: 13,301
Trainable params: 13,301
Non-trainable params: 0
______________________________________________________

259/259 [==============================] - 0s 938us/sample - loss: 50.8207 - mean_absolute_error: 28.6362 - val_loss: 58.8573 - val_mean_absolute_error: 37.2100
Epoch 42/100
259/259 [==============================] - 0s 942us/sample - loss: 72.7690 - mean_absolute_error: 50.5836 - val_loss: 47.2165 - val_mean_absolute_error: 25.5162
Epoch 43/100
259/259 [==============================] - 0s 938us/sample - loss: 47.7354 - mean_absolute_error: 25.5670 - val_loss: 30.4702 - val_mean_absolute_error: 8.8230
Epoch 44/100
259/259 [==============================] - 0s 911us/sample - loss: 48.8494 - mean_absolute_error: 26.6929 - val_loss: 49.3611 - val_mean_absolute_error: 27.6836
Epoch 45/100
259/259 [==============================] - 0s 913us/sample - loss: 56.7795 - mean_absolute_error: 34.6447 - val_loss: 137.0741 - val_mean_absolute_error: 115.7558
Epoch 46/100
259/259 [==============================] - 0s 915us/sample - loss: 52.3817 - mean_absolute_error: 30.2572 - val_loss: 65.3937 - v

259/259 [==============================] - 0s 1ms/sample - loss: 46.7579 - mean_absolute_error: 25.1296 - val_loss: 40.5153 - val_mean_absolute_error: 19.4389
Epoch 89/100
259/259 [==============================] - 0s 1ms/sample - loss: 52.0776 - mean_absolute_error: 30.4724 - val_loss: 47.7158 - val_mean_absolute_error: 26.6817
Epoch 90/100
259/259 [==============================] - 0s 1ms/sample - loss: 57.4767 - mean_absolute_error: 35.8734 - val_loss: 86.2939 - val_mean_absolute_error: 65.2617
Epoch 91/100
259/259 [==============================] - 0s 1ms/sample - loss: 66.6222 - mean_absolute_error: 45.0303 - val_loss: 85.5347 - val_mean_absolute_error: 64.3095
Epoch 92/100
259/259 [==============================] - 0s 1ms/sample - loss: 48.2756 - mean_absolute_error: 26.6829 - val_loss: 44.1214 - val_mean_absolute_error: 23.1159
Epoch 93/100
259/259 [==============================] - 0s 1ms/sample - loss: 46.4495 - mean_absolute_error: 24.8760 - val_loss: 46.3185 - val_mean_absol

In [51]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model.predict(cen_x_train)
print(np.sqrt(mean_squared_error(cen_y_train,pred_train)))
pred = model.predict(cen_x_test)
print(np.sqrt(mean_squared_error(cen_y_test,pred)))

pres_train = pd.DataFrame(pred_train)
pres_train.columns = ['Prediction']
cen_y_train = pd.DataFrame(cen_y_train)
cen_y_train.columns = ['Real Value']
results = pd.concat([pres_train, cen_y_train], axis = 1)
results

12.03919319410426
131.7876836988274


,Prediction,Real Value
0,7464.045410,7474.33949
1,5879.779785,5895.00295
2,5316.462891,5324.94201
3,7742.329102,7739.26681
4,7769.764160,7747.53124
...,...,...
283,6094.388672,6105.05688
284,6091.633789,6075.02433
285,7419.895508,7421.35666
286,6666.653320,6662.96869
